In [1]:
import psycopg2
import pandas as pd
import configparser
from psycopg2 import sql

In [2]:
config= configparser.ConfigParser()
config.read_file(open('credentials.cfg'))

In [3]:
host=config.get('REDSHIFT','HOST')
port = config.get('REDSHIFT','PORT')
database = config.get('REDSHIFT','DATABASE')
user = config.get('REDSHIFT','USER')
password = config.get('REDSHIFT','PASSWORD')


In [4]:
print(host)

nepsedata.404951734109.ap-south-1.redshift-serverless.amazonaws.com


In [6]:
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
cursor = conn.cursor()


In [6]:

cursor.execute("SELECT * FROM information_schema.tables WHERE table_name='nepse';")
table_exits = bool(cursor.rowcount)
if not table_exits:
    cursor.execute("CREATE TABLE IF NOT EXISTS nepse(company_name varchar(100), date varchar(100), confidence float, open_price float,highest_price float,lowest_price float, closing_price float, VWAP float, total_traded_quantity float, Previous_closing float, total_traded_value float, total_trades float, difference float, range float, difference_percentage float, range_percentage float, VWAP_percentage float, year_high float, year_low float)")
    conn.commit()


In [60]:
conn.commit ()

In [7]:
df = pd.read_csv('nepsedata.csv')
data = [tuple(row) for row in df.values]
sql = 'INSERT INTO nepse VALUES %s'
args_str = ','.join(cursor.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", x).decode('utf-8') for x in data)
cursor.execute(sql % args_str)
conn.commit()

In [7]:
cursor.execute("SELECT count(*) FROM nepse")
print(cursor.fetchall())

[(75441,)]


In [ ]:
### find